In [17]:

print("Torch version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


Torch version: 2.5.1+cu121
CUDA Available: True
CUDA Version: 12.1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [18]:
import torch.nn as nn
import torch
import torch.optim as optim
from torchvision import datasets,transforms
from torchvision.utils import save_image
import os

batch_size = 128
z_dim = 100
num_classes = 10
img_size = 28
channels = 1
epochs = 50
lr = 0.0002
beta1 = 0.5

# Create output folder
os.makedirs("DC_GAN_DC_GAN_DC_GAN_DC_GAN_DC_GAN_generated_images", exist_ok=True)

In [19]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.',train=True,download = True,transform = transform),
    batch_size = batch_size,
    shuffle = True
)

In [20]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: (N, z_dim, 1, 1)
            nn.ConvTranspose2d(z_dim, 256, 7, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):  # ✅ Correct order
        return self.net(z)


In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.net = nn.Sequential(
            # Input = (N,1,28,28)
            nn.Conv2d(1,64,4,2,1,bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64,128,4,2,1,bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Flatten(),
            nn.Linear(128*7*7,1),
            nn.Sigmoid(),
            
        )
    def forward(self,img):
        return self.net(img)

In [22]:
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr = lr , betas = (beta1,0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr = lr , betas = (beta1,0.999))

In [23]:
k = 3  # Generator updates per iteration
p = 1  # Discriminator updates per iteration

In [24]:
for epoch in range(1,epochs+1):
    for i , (real_imgs,_) in enumerate(train_loader):
        batch_size_curr = real_imgs.size(0)
        real_imgs = real_imgs.to(device)
        real = torch.ones(batch_size_curr,1,device = device)
        fake = torch.zeros(batch_size_curr, 1, device=device)
        
        ################# Train Discriminator p times ###############################

        for _ in range(p):
            z = torch.randn(batch_size_curr , z_dim , 1 ,1 , device = device)
            fake_imgs = generator(z)

            # Real

            real_validity = discriminator(real_imgs)
            d_real_loss = criterion(real_validity,real)

            # Fake 

            fake_validity = discriminator(fake_imgs.detach())
            d_fake_loss = criterion(fake_validity, fake)

            d_loss = d_real_loss + d_fake_loss

            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()

        ######################## Train Generator k times ##########################

        for _ in range(k):
            z = torch.randn(batch_size_curr , z_dim , 1 , 1 , device = device)
            fake_imgs = generator(z)
            validity = discriminator(fake_imgs)
            g_loss = criterion(validity , real) # Fool D -> label as real 
            optimizer_G.zero_grad()

        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

        # Save sample images

        generator.eval()
        with torch.no_grad():
            z = torch.randn(64 , z_dim , 1 , 1 , device = device)
            samples = generator(z)
            samples = samples*0.5 + 0.5 # Denormalize
            save_image(samples, f"DC_GAN_generated_images/epoch_{epoch}.png", nrow=8)
        generator.train()

[Epoch 1/50] [Batch 0/469] D Loss: 1.3919 | G Loss: 1.0150
[Epoch 1/50] [Batch 200/469] D Loss: 0.0007 | G Loss: 8.0134
[Epoch 1/50] [Batch 400/469] D Loss: 0.0002 | G Loss: 9.1675
[Epoch 2/50] [Batch 0/469] D Loss: 0.0002 | G Loss: 9.6181
[Epoch 2/50] [Batch 200/469] D Loss: 0.0001 | G Loss: 10.1476
[Epoch 2/50] [Batch 400/469] D Loss: 0.0000 | G Loss: 10.5543
[Epoch 3/50] [Batch 0/469] D Loss: 0.0000 | G Loss: 10.7302
[Epoch 3/50] [Batch 200/469] D Loss: 0.0000 | G Loss: 11.1469
[Epoch 3/50] [Batch 400/469] D Loss: 0.0000 | G Loss: 11.4772
[Epoch 4/50] [Batch 0/469] D Loss: 0.0000 | G Loss: 11.5145
[Epoch 4/50] [Batch 200/469] D Loss: 0.0000 | G Loss: 11.8484
[Epoch 4/50] [Batch 400/469] D Loss: 0.0000 | G Loss: 12.0940
[Epoch 5/50] [Batch 0/469] D Loss: 0.0000 | G Loss: 12.1541
[Epoch 5/50] [Batch 200/469] D Loss: 0.0000 | G Loss: 12.2535
[Epoch 5/50] [Batch 400/469] D Loss: 0.0000 | G Loss: 12.5011
[Epoch 6/50] [Batch 0/469] D Loss: 0.0000 | G Loss: 12.6127


KeyboardInterrupt: 

In [ ]:

import os
from torchvision.utils import save_image

def generate_images_dcgan_individually(generator, num_samples=512, save_dir="DC_GAN_generated_images", prefix="img"):
    
    generator.eval()  # Set generator to evaluation mode
    z = torch.randn(num_samples, z_dim, 1, 1, device=device)  # Sample random noise
    os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist
    with torch.no_grad():  # Disable gradient computation for inference
        gen_imgs = generator(z)  # Generate images
        gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize images to [0, 1]
        for i, img in enumerate(gen_imgs):
            save_path = os.path.join(save_dir, f"{prefix}_{i:04d}.png")
            save_image(img, save_path)  # Save each image individually


In [26]:

generate_images_dcgan_individually(generator, num_samples=10)